## Overfitting
- underfitting : 학습 결과가 데이터를 제대로 설명하지 못함
- overfitting : 너무 학습을 많이 한 나머지 데이터를 외워 외부 데이터에 대한 예측을 제대로 하지 못함

train dataset에서는 high accuracy지만, 실제 분류가 필요한 test accuracy 는 낮은 현상. 

### sol
- more training data
- reduce the number of features
- regularization
- dropout

## Dropout
학습을 진행하면서 각 레이어에 존재하는 뉴런들을 무작위로 껐다 켰다 반복함. 
- 각 켜진 노드들만 각 레이어를 통과하고 최종 output과 GT와의 loss를 이용해 역전파로 네트워크를 업데이트 함. 
- overfitting을 줄일 수 있음!

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [11]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100
drop_prob = 0.3

In [3]:
mnist_train = dsets.MNIST(root='MNIST_data/', 
                          train = True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [4]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

##### drop out 적용

In [5]:
linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear4 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10, bias=True)

relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

In [6]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0565,  0.0423, -0.0155,  ...,  0.1012,  0.0459, -0.0191],
        [ 0.0772,  0.0452, -0.0638,  ...,  0.0476, -0.0638,  0.0528],
        [ 0.0311, -0.1023, -0.0701,  ...,  0.0412, -0.1004,  0.0738],
        ...,
        [ 0.0334,  0.0187, -0.1021,  ...,  0.0280, -0.0583, -0.1018],
        [-0.0506, -0.0939, -0.0467,  ..., -0.0554, -0.0325,  0.0640],
        [-0.0183, -0.0123,  0.1025,  ..., -0.0214,  0.0220, -0.0741]],
       requires_grad=True)

In [7]:
model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4, relu, dropout,
                            linear5).to(device)

In [8]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_batch = len(data_loader)
model.train() # set the model to train mode (dropout = True)
# model.train() : dropout을 사용하겠다!!!
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.314410478
Epoch: 0002 cost = 0.145182982
Epoch: 0003 cost = 0.114784971
Epoch: 0004 cost = 0.092163585
Epoch: 0005 cost = 0.087830812
Epoch: 0006 cost = 0.074759237
Epoch: 0007 cost = 0.068311185
Epoch: 0008 cost = 0.060030185
Epoch: 0009 cost = 0.060201362
Epoch: 0010 cost = 0.055767026
Epoch: 0011 cost = 0.051428664
Epoch: 0012 cost = 0.047851596
Epoch: 0013 cost = 0.049270045
Epoch: 0014 cost = 0.043954659
Epoch: 0015 cost = 0.044984676
Learning finished


In [ ]:
# Test model and check accuracy
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)
    # model.eval() : dropout 끄기

    # Test the model using test sets
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9797999858856201
Label:  8
Prediction:  8


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\datasets\mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\datasets\mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
